In [2]:
import pandas as pd
import pickle
import numpy as np
import datetime 
from os.path import join as pjoin
import os
#import argparse
#import yamlb

In [3]:
tmp_data_path =  './data/tmp'
s_year = 1997
e_year = 2020

# import data

3 groups of data
- bridge 1: wrds bridge
- bridge 2: evans bridge
- SDC MA data

In [17]:
# bridge 1
wrds_bridge = pd.read_csv('./data/wrds_bridge.csv', header=0, engine=None)

In [18]:
# rename 

wrds_bridge.rename(columns={x: x.replace(' ', '_').upper().strip() for x in wrds_bridge.columns}, inplace=True)

In [19]:
# for now, we just need 4 columns

wrds_bridge = wrds_bridge[['CUSIP', 'GVKEY', 'LINKDT', 'LINKENDDT', 'CONM']]

In [20]:
# see data structure
wrds_bridge.sample(3)


,CUSIP,GVKEY,LINKDT,LINKENDDT,CONM
22780,05459C108,62690,04/24/1996,12/29/2000,AXENT TECHNOLOGIES INC
22640,P36209107,62437,04/30/1996,01/27/2006,ELAMEX SA DE CV
18905,03814L103,28740,09/30/1993,12/12/2000,APPLIED BIOMETRICS INC


In [21]:
# bridge 2
evans_bridge = pd.read_csv('./data/evans_bridge.csv')

In [22]:
evans_bridge.sample(3)

,DealNumber,agvkey,tgvkey
119093,2725271040,NaN,113490.0
13898,690558020,25533.0,NaN
50588,1349614020,10247.0,28930.0


In [23]:
# AF - BJ

name_lst = [
    'ACU', 'ASIC2', 'ABL', 'ANL', 'APUBC', 'AUP', 'AUPSIC', 'AUPBL', 'AUPNAMES', 'AUPPUB',
    'BLOCK','CREEP','DA','DE','STATC','SYNOP','VAL','PCTACQ','PSOUGHTOWN','PSOUGHT','PHDA','PCTOWN','PSOUGHTT','PRIVATIZATION','DEAL_NO',
    'TCU', 'TSIC2', 'TBL', 'TNL', 'TPUBC', 'TUP', 'TUPSIC', 'TUPBL', 'TUPNAMES', 'TUPPUB'    
]

before concat all data, manually convert all date on excel file to "YY-mm-dd" format.

Guide: select all date var --> right-click --> Cell format --> Date --> 2012-03-14 --> OK

![](https://cdn.mathpix.com/snip/images/58hcVJ3qFlC446Ns4SaJKtm-UroEqUKyqu4oCnTWhKY.original.fullsize.png)

In [16]:
def concat_data(st, end):
    df_l = []
    for year in range(st, end+1, 1):
        df = pd.read_excel(f"./data/SDC/{year}.xlsx", header=1, engine='openpyxl')
        #df = df.drop(df.columns[4], axis=1) # this column is duplicate with column 3 
        #print(len(df.columns))
        df.columns = name_lst
        
        # check date var loading ok
        check = df[df['DA'] == datetime.time(0, 0)]
        if check.shape[0] == 0 :
            print('date variables loading ok \n')
        else:
            print('date variables loading fail, please manually check. number of failed records: ', check.shape[0])
        
        df_l.append(df)
        print(f'{year} data shape:', df.shape)
        del df
    df = pd.concat(df_l)
    return df

In [25]:
# You can run code start from here if merging sdf_df is unnecessary

if os.path.isfile(pjoin(tmp_data_path , f'sdc_{s_year}_{e_year}.pickle')):
    sdc_df = pd.read_pickle(pjoin(tmp_data_path , f'sdc_{s_year}_{e_year}.pickle'))
else:
    sdc_df = concat_data(s_year, e_year)
    sdc_df = sdc_df.reset_index(drop=True) 
    print(f'saving sdc table ranging from {s_year} to {e_year} ...')
    sdc_df.to_pickle(pjoin(tmp_data_path , f'sdc_{s_year}_{e_year}.pickle'))

## Data Preparing

1. delete obs either `ACU` or `TCU` in full_df is NAs (otherwise we cannot identify the participants of deals)
2. match the format of: time, CUSIP, GVKEY to be the same across three dataset (otherwise may trigger error when merging tables)
    - Time: Timestap
    - CUSIP / GVKEY: string
    - dealnum: string



###  sdc data

1. drop where either `ACU` or `TCU` is Nas
1. fill DEAL_NO NAs to -1
1. change all identifier to `str`; including: `ACU`, `AUP`, `TCU`, `TUP`, `DEAL_NO`, `GVKEY`a

In [26]:
sdc_df = sdc_df.dropna(subset=['ACU','TCU']) # actually nothing drops
# deal num
sdc_df['DEAL_NO'] = sdc_df['DEAL_NO'].fillna(-1)
sdc_df['DEAL_NO'] = sdc_df['DEAL_NO'].astype(str)
sdc_df['TCU']  = sdc_df['TCU'].astype('str')
sdc_df['ACU']  = sdc_df['ACU'].astype('str')
sdc_df['TUP']  = sdc_df['TUP'].astype('str')
sdc_df['AUP']  = sdc_df['AUP'].astype('str')

### bridges data

1. match variable type for merging
    1. for CUSIP/GVKEY/DEALNUM, all convert to `string`; 
        - do not keep 0s at front 
            - e.g. `002030` will be curtail to `2030`
    2. for time, all convert to pandas `Timestamp` instance
2. drop na or fill na


#### evans_bridge

for evans_brdge, load as float:
1. fill na as -1
2. convert all var to integer
3. convert all var to string


**so the GVKEY has no 0 at front**

In [27]:
evans_bridge.DealNumber = evans_bridge.DealNumber.fillna(-1)
evans_bridge.tgvkey = evans_bridge.tgvkey.fillna(-1)
evans_bridge.agvkey = evans_bridge.agvkey.fillna(-1)


evans_bridge = evans_bridge.astype('int')
evans_bridge = evans_bridge.astype('str')


#### wrds_bridge

- `GVKEY` and `CUSIP`, load as int; so just convert to str
- no need to worry about NA

In [28]:
wrds_bridge[['GVKEY', 'CUSIP']] = wrds_bridge[['GVKEY', 'CUSIP']].applymap(str)

In [29]:
# covvert 9 digit to 6 digit; 
if len(wrds_bridge.CUSIP[0]) == 9:
    wrds_bridge['CUSIP'] = wrds_bridge.apply(lambda row: row['CUSIP'][:6], axis=1)

since there are lots of "E" in LINKENDDT(still effective presently), LINKENDDT was read as str;

we replace "E" as '12/31/2099' (string type), and then convert to pandas timestamp

In [30]:
wrds_bridge.LINKENDDT[wrds_bridge.LINKENDDT == 'E'] = '12/31/2099'
wrds_bridge['LINKENDDT'] = pd.to_datetime(wrds_bridge['LINKENDDT'],format = '%m/%d/%Y' )
wrds_bridge['LINKDT'] = pd.to_datetime(wrds_bridge['LINKDT'])

# Types of Takeover


## Public?
- Although `GVKEY` is mostly for public firm, but some private firm do have `GVKEY` as well.
- However, TNIC would only cover Public firms.

Which means, we could only analysis the following conditions:

| APUBC == Public? | AUPPUB == Public? | TPUBC == Public? | TUPPUB == Public? | mark as                                           |
|------------------|-------------------|------------------|-------------------|---------------------------------------------------|
| 1                | 1                 | 1                | 1                 | 1                                                 |
| 1                | 1                 | 1                | 0                 | 2                                                 |
| 1                | 1                 | 0                | 1                 | 3                                                 |
| 1                | 0                 | 1                | 1                 | 4                                                 |
| 1                | 0                 | 1                | 0                 | 5                                                 |
| 1                | 0                 | 0                | 1                 | 6                                                 |
| 0                | 1                 | 1                | 1                 | 7                                                 |
| 0                | 1                 | 1                | 0                 | 8                                                 |
| 0                | 1                 | 0                | 1                 | 9                                                 |
|                  |                   |                  |                   | all other combination is unanalysiable, mark as 0 |

The analysis the ratio refer to [Appendix 1](./Appendix1_data_explore.ipynb)

**Conclusion**
- `3` and `9` are one type of MA
- `1` and `7` are another type of MA
- drop others






In [31]:
def mark_if_public(row, **kwargs):
    if (row.APUBC == 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB == 'Public'):
        return '1'
    elif (row.APUBC == 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB != 'Public'):
        return '2'
    elif (row.APUBC == 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC != 'Public') & (row.TUPPUB == 'Public'):
        return '3'
    elif (row.APUBC == 'Public') &  (row.AUPPUB != 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB == 'Public'):
        return '4'
    elif (row.APUBC == 'Public') &  (row.AUPPUB != 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB != 'Public'):
        return '5'
    elif (row.APUBC == 'Public') &  (row.AUPPUB != 'Public') & (row.TPUBC != 'Public') & (row.TUPPUB == 'Public'):
        return '6'
    elif (row.APUBC != 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB == 'Public'):
        return '7'
    elif (row.APUBC != 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB != 'Public'):
        return '8'
    elif (row.APUBC != 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC != 'Public') & (row.TUPPUB == 'Public'):
        return '9'
    else:
        return '0'

In [32]:
sdc_df['PUB_COND'] = sdc_df.apply(mark_if_public, axis=1)

In [34]:
# sdc_df_public =sdc_df[sdc_df['PUB_COND'].isin(['1','3','7','9'])].copy()

In [35]:
def save_df_to_pickle(df, name):
    print(f'pickle data shape = ', df.shape)
    df.to_pickle(pjoin(tmp_data_path , f'{name}_{s_year}_{e_year}.pickle'))
    

In [36]:
# print(f'pickle merged data with public info from {s_year} to {e_year} = ',sdc_df_public.shape)
# sdc_df_public.to_pickle(pjoin(tmp_data_path , f'sdc_df_public_{s_year}_{e_year}.pickle'))

pickle merged data with public info from 1997 to 2020 =  (48867, 36)


In [87]:
name_lst += ['PUB_COND']

# STATC 

In [40]:
sdc_df.STATC.value_counts()

C     226123
P      24031
I      11278
W       6036
UN      2934
S       2243
DR       227
SW       162
IW        66
R         38
PC         8
L          3
U          1
Name: STATC, dtype: int64

In [44]:
sdc_df_comp=sdc_df[sdc_df.STATC.isin(['C'])].copy()

In [45]:
save_df_to_pickle(sdc_df_comp, 'sdc_public_statc')

pickle data shape =  (226123, 36)


# Merge GVKEY

In [47]:
#before merge, check all merge variables are in the same type

# gvkey, cusip match

assert type(wrds_bridge.GVKEY[0]) == type(wrds_bridge.CUSIP[0]) == type(evans_bridge.agvkey[0]) 

assert type(evans_bridge.agvkey[0]) == type(evans_bridge.tgvkey[0]) == type(sdc_df_comp.ACU[0]) == type(sdc_df_comp.TCU[0])
        
# deal number match

assert type(sdc_df_comp.DEAL_NO[0]) == type(evans_bridge.DealNumber[0])

# time match

assert type(sdc_df_comp.DA[0]) == type(sdc_df_comp.DE[0]) == type(wrds_bridge.LINKDT[0]) == type(wrds_bridge.LINKENDDT[0])
    
    

## obtain GVKEY from linkings

Rules:
1. use WRDS linking table as primary table, EVANS as secondary
2. first, use ACU and TCU to search GVKEY; If no result return, use AUP and TUP to search. If still no result return, this row has to be dropped.


reason for rules refering to [Appendix 1](./Appendix1_data_explore.ipynb)

+ save merged data to pickle before checking out Appendix 1!

In [48]:
# 1st, take care of Acquiror
merged_w_a = sdc_df_comp.merge(wrds_bridge, left_on='ACU', right_on = 'CUSIP', how = 'left')
merged_w_t = merged_w_a.merge(wrds_bridge, left_on='TCU', right_on = 'CUSIP', how = 'left')
merged_w_ua = merged_w_t.merge(wrds_bridge, left_on='AUP', right_on = 'CUSIP', how = 'left')
merged_w_ut = merged_w_ua.merge(wrds_bridge, left_on='TUP', right_on = 'CUSIP', how = 'left')



## Remove self merge

some time self merge self

In [53]:
merged_w_ut = merged_w_ut[merged_w_ut.ACU != merged_w_ut.TCU]

In [56]:
merged_w_ut = merged_w_ut.reset_index(drop=True)

## Filter those Gvkey condition not ok

In [57]:
merged_w_ut.columns

Index(['ACU', 'ASIC2', 'ABL', 'ANL', 'APUBC', 'AUP', 'AUPSIC', 'AUPBL',
       'AUPNAMES', 'AUPPUB', 'BLOCK', 'CREEP', 'DA', 'DE', 'STATC', 'SYNOP',
       'VAL', 'PCTACQ', 'PSOUGHTOWN', 'PSOUGHT', 'PHDA', 'PCTOWN', 'PSOUGHTT',
       'PRIVATIZATION', 'DEAL_NO', 'TCU', 'TSIC2', 'TBL', 'TNL', 'TPUBC',
       'TUP', 'TUPSIC', 'TUPBL', 'TUPNAMES', 'TUPPUB', 'PUB_COND', 'CUSIP_x',
       'GVKEY_x', 'LINKDT_x', 'LINKENDDT_x', 'CONM_x', 'CUSIP_y', 'GVKEY_y',
       'LINKDT_y', 'LINKENDDT_y', 'CONM_y', 'CUSIP_x', 'GVKEY_x', 'LINKDT_x',
       'LINKENDDT_x', 'CONM_x', 'CUSIP_y', 'GVKEY_y', 'LINKDT_y',
       'LINKENDDT_y', 'CONM_y'],
      dtype='object')

In [51]:
wrds_bridge.columns

Index(['CUSIP', 'GVKEY', 'LINKDT', 'LINKENDDT', 'CONM'], dtype='object')

In [79]:
gvkey_name_list = []

In [80]:
for key in ['ACU','TCU','AUP','TUP']:
    lst = [name+"_"+key for name in ['CUSIP', 'GVKEY', 'LINKDT', 'LINKENDDT', 'CONM'] ]
#    print(len(lst))
    gvkey_name_list += lst

In [82]:
assert len(gvkey_name_list) == 20

In [88]:
merged_name_list = name_lst + gvkey_name_list

In [89]:
merged_raw = merged_w_ut.copy()

In [91]:
merged_raw.columns = merged_name_list

### filtering:

the following conditions are marked as GVKEY merged successfully:

`ok = (GVKEY Found in Bridge table) & (GVKEY in valid time period)`

num of succcess condition = (C22 + C21) * (C22 + C21) = 9

| ACU ok | AUP  ok | TCU ok | TUP ok | mark as                                           |
|------------------|-------------------|------------------|-------------------|---------------------------------------------------|
| 1                | 1                 | 1                | 1                 | 1                                                 |
| 1                | 1                 | 1                | 0                 | 2                                                 |
| 1                | 1                 | 0                | 1                 | 3                                                 |
| 1                | 0                 | 1                | 1                 | 4                                                 |
| 1                | 0                 | 1                | 0                 | 5                                                 |
| 1                | 0                 | 0                | 1                 | 6                                                 |
| 0                | 1                 | 1                | 1                 | 7                                                 |
| 0                | 1                 | 1                | 0                 | 8                                                 |
| 0                | 1                 | 0                | 1                 | 9                                                 |
|                  |                   |                  |                   | all other combination is unanalysiable, mark as 0 |


1. Target and Acquirer must at least successfully match with one table
1. for those records (acquirer or target) that evans bridge did not much successfully, but wrds bridge matched successfully, confirm the `date accounced` of the deal falls in the effective time period of the linking. 

    why use `DA` instead of `DE`? since `DA` has less Nas
    
    
* I chose to use wrds to be my primary linking. If two linking both matched, I will choose the result from wrds.


In [113]:
def mark_gvkey_ok(row, key):
    '''
    
    '''
#    print(row['GVKEY_'+key])
    if pd.notna(row['GVKEY_'+key]) & (row['LINKDT_'+key] <= row['DA']) & (row['LINKENDDT_'+key] >= row['DA']):
        return 1
    else:
        return 0
    
    

In [114]:
for part in ['A', 'T']:
    for ent in ['CU', 'UP']:
        key = part+ent
        merged_raw[key+'_OK'] = merged_raw.apply(mark_gvkey_ok, key=key, axis=1)

here I simplify the condition.

Since if the GVKEY of direct participants are exist, we will use the GVKEY of them instead of the GVKEY of their ultimate parents.

So, the simplier version of the gvkey condition is:

| ACU ok | AUP  ok | TCU ok | TUP ok | mark as                                           |
|------------------|-------------------|------------------|-------------------|---------------------------------------------------|
| 1                | 1                 | 1                | 1                 | 1                                                 |
| 1                | 1                 | 1                | 0                 | 1                                                 |
| 1                | 1                 | 0                | 1                 | 3                                                 |
| 1                | 0                 | 1                | 1                 | 1                                                 |
| 1                | 0                 | 1                | 0                 | 1                                                 |
| 1                | 0                 | 0                | 1                 | 3                                                 |
| 0                | 1                 | 1                | 1                 | 2                                                 |
| 0                | 1                 | 1                | 0                 | 2                                                 |
| 0                | 1                 | 0                | 1                 | 4                                                 |
|                  |                   |                  |                   | all other combination is unanalysiable, mark as 0 |



In [125]:
def mark_gvkey_total_ok(row):
    
    if row.ACU_OK & row.TCU_OK:
        return '1'
    elif row.AUP_OK & row.TCU_OK:
        return '2'
    elif row.ACU_OK & row.TUP_OK:
        return '3'
    elif row.AUP_OK & row.TUP_OK:
        return '4'
    else:
        return '0'
        
        

In [126]:
merged_raw['GVKEY_OVERALL'] = merged_raw.apply(mark_gvkey_total_ok, axis=1)

In [127]:
merged_raw['GVKEY_OVERALL'].value_counts()

0    273072
3      6784
1      4938
4      3415
2      1625
Name: GVKEY_OVERALL, dtype: int64

In [128]:
merged_raw.to_pickle(pjoin(tmp_data_path , f'max_master1_{s_year}_{e_year}.pickle'))

# only keep some var

merged_raw['GVKEY_OVERALL'] must be [1,2,3,4]; and drop other help variables (Only contain name_lst + AGVKEY + TGVKEY)


In [132]:
def gvkey_filter_a(row):
    if row['GVKEY_OVERALL'] in ['1', '3']:
        return row['GVKEY_'+'ACU']
    elif row['GVKEY_OVERALL'] in ['2', '4']:
        return row['GVKEY_'+'AUP']
    
    
def gvkey_filter_t(row):
    if row['GVKEY_OVERALL'] in ['1', '2']:
        return row['GVKEY_'+'TCU']
    elif row['GVKEY_OVERALL'] in ['3', '4']:
        return row['GVKEY_'+'TUP']

        
    

In [145]:
merged_raw_no0 = merged_raw[merged_raw['GVKEY_OVERALL'] != '0' ].copy()

In [146]:
merged_raw_no0['AGVKEY'] = merged_raw_no0.apply(gvkey_filter_a, axis=1)

In [147]:
merged_raw_no0['TGVKEY'] = merged_raw_no0.apply(gvkey_filter_t, axis=1)

In [144]:
keep_lst = [
    'ACU', 'ASIC2', 'ABL', 'ANL', 'APUBC', 'AUP', 'AUPSIC', 'AUPBL', 'AUPNAMES', 'AUPPUB',
    'BLOCK','CREEP','DA','DE','STATC','SYNOP','VAL','PCTACQ','PSOUGHTOWN','PSOUGHT','PHDA','PCTOWN','PSOUGHTT','PRIVATIZATION','DEAL_NO',
    'TCU', 'TSIC2', 'TBL', 'TNL', 'TPUBC', 'TUP', 'TUPSIC', 'TUPBL', 'TUPNAMES', 'TUPPUB',
    'AGVKEY', 'TGVKEY','GVKEY_OVERALL'
] 

In [148]:
merged = merged_raw_no0[keep_lst]

In [150]:
merged.shape

(16762, 38)

# Add 2 variables SIC and YEAR

In [152]:
def get_sic(df):
    '''
    df: the sdc table contains sic variable named as `ASIC2`
    
    '''
    x = df.ASIC2.str.split('/')
    x = x.transform(lambda x: x[0] if not isinstance(x, float) else np.nan)
    df['SIC_A'] = x

    x = df.ASIC2.str.split('/')
    x = x.transform(lambda x: x[0] if not isinstance(x, float) else np.nan)
    df['SIC_T'] = x
    
    return df

In [154]:
merged["YEAR"] = merged.DA.dt.year

<ipython-input-154-227b9d1f1244>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["YEAR"] = merged.DA.dt.year


In [155]:
merged = get_sic(merged)

<ipython-input-152-fd17d6ebba7f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SIC_A'] = x
<ipython-input-152-fd17d6ebba7f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SIC_T'] = x


In [157]:
merged.to_pickle(pjoin(tmp_data_path , f'master1_{s_year}_{e_year}.pickle'))